<a href="https://colab.research.google.com/github/lucifernandes/acidentesPRF/blob/master/scrub_explore_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dados abertos sobre Acidentes da Polícia Rodoviária Federal

Lendo as bases de dados

In [2]:
import pandas as pd

url = 'https://media.githubusercontent.com/media/lucifernandes/acidentesPRF/master/Bases'
bases = {'Ocorrencias': {'folder': 'Agrupados%20por%20ocorrencia', 
                         'files': ['datatran2007.csv', 'datatran2008.csv', 'datatran2009.csv', 'datatran2010.csv', 'datatran2011.csv', 'datatran2012.csv', 'datatran2013.csv', 'datatran2014.csv', 'datatran2015.csv', 'datatran2016_atual.csv', 'datatran2017.csv', 'datatran2018.csv', 'datatran2019.csv', 'datatran2020.csv']}, 
         'Pessoas': {'folder': 'Agrupados%20por%20pessoa', 
                     'files': ['acidentes2007.csv', 'acidentes2008.csv', 'acidentes2009.csv', 'acidentes2010.csv', 'acidentes2011.csv', 'acidentes2012.csv', 'acidentes2013.csv', 'acidentes2014.csv', 'acidentes2015.csv', 'acidentes2016_atual.csv', 'acidentes2017.csv', 'acidentes2018.csv', 'acidentes2019.csv', 'acidentes2020.csv']}, 
         'Acidentes': {'folder': 'Agrupados%20por%20pessoa%20-%20Todas%20as%20causas%20e%20tipos%20de%20acidentes%20(desde%202017)', 
                       'files': ['acidentes2017_todas_causas_tipos.csv', 'acidentes2018_todas_causas_tipos.csv', 'acidentes2019_todas_causas_tipos.csv', 'acidentes2020_todas_causas_tipos.csv']}
         }

In [ ]:
# Mostra o link de todas as bases
for base in bases.keys():
  for arq in bases[base]['files']:
    print(f'{url}/{bases[base]["folder"]}/{arq}?raw=true')

In [ ]:
df = pd.read_csv(f'{url}/{bases["Ocorrencias"]["folder"]}/{bases["Ocorrencias"]["files"][0]}?raw=true', encoding='ISO-8859-1', sep=';')
#df = pd.read_csv(f'{url}/{bases["Pessoas"]["folder"]}/{bases["Pessoas"]["files"][0]}?raw=true', encoding='ISO-8859-1', sep=',')
#df = pd.read_csv(f'{url}/{bases["Acidentes"]["folder"]}/{bases["Acidentes"]["files"][0]}?raw=true', encoding='ISO-8859-1', sep=';')

df.info()

# **Etapa Limpeza**

Para a etapa de limpeza, as bases de dados foram estudadas separadamente, como será mostrado a seguir:

In [ ]:
# LUCAS
dfOcorrencias = pd.read_csv(f'{url}/{bases["Ocorrencias"]["folder"]}/{bases["Ocorrencias"]["files"][0]}?raw=true', encoding='ISO-8859-1', sep=';')
dfPessoas = pd.read_csv(f'{url}/{bases["Pessoas"]["folder"]}/{bases["Pessoas"]["files"][0]}?raw=true', encoding='ISO-8859-1', sep=',')

In [ ]:
# Informações
print(f'OCORRÊNCIAS: {bases["Ocorrencias"]["files"][0]}')
print(f'Qtd de IDs: {len(dfOcorrencias.id)}')
print(f'Qtd de IDs repetidos: {len(dfOcorrencias.id) - len(dfOcorrencias.id.unique())}')
print(f'Qtd de BR igual a (null): {len(dfOcorrencias[dfOcorrencias.br == "(null)"])}')
print(f'Qtd de UF igual a (null): {len(dfOcorrencias[dfOcorrencias.uf == "(null)"])}')
print(f'Qtd de Classificação de Acidentes igual a (null): {len(dfOcorrencias[dfOcorrencias.classificacao_acidente == "(null)"])}')
print(f'Qtd de Fase do Dia igual a (null): {len(dfOcorrencias[dfOcorrencias.fase_dia == "(null)"])}')

print(f'PESSOAS: {bases["Pessoas"]["files"][0]}')
print(f'Qtd de IDs: {len(dfPessoas.id)}')
print(f'Qtd de IDs repetidos: {len(dfPessoas.id) - len(dfPessoas.id.unique())}')
print(f'Qtd de BR igual a (null): {len(dfPessoas[dfPessoas.br == "(null)"])}')
print(f'Qtd de UF igual a (null): {len(dfPessoas[dfPessoas.uf == "(null)"])}')
print(f'Qtd de Classificação de Acidentes igual a (null): {len(dfPessoas[dfPessoas.classificacao_acidente == "(null)"])}')
print(f'Qtd de Fase do Dia igual a (null): {len(dfPessoas[dfPessoas.fase_dia == "(null)"])}')
print(f'Qtd de ID do Veículo igual a (null): {len(dfPessoas[dfPessoas.id_veiculo == "(null)"])}')
print(f'Qtd de Tipo do Veículo igual a (null): {len(dfPessoas[dfPessoas.tipo_veiculo == "(null)"])}')
print(f'Qtd de Marca igual a (null): {len(dfPessoas[dfPessoas.marca == "(null)"])}')
print(f'Qtd de Ano de Fabricação igual a (null): {len(dfPessoas[dfPessoas.ano_fabricacao_veiculo == "(null)"])}')
print(f'Qtd de Idade igual a NA: {len(dfPessoas[dfPessoas.idade == "NA"])}')
print(f'Qtd de Sexo igual a NA: {len(dfPessoas[dfPessoas.sexo == "NA"])}')
print(f'Qtd de Nacionalidade igual a NA: {len(dfPessoas[dfPessoas.nacionalidade == "NA"])}')
print(f'Qtd de Naturalidade igual a NA: {len(dfPessoas[dfPessoas.naturalidade == "NA"])}')

In [ ]:
#Angela

#Obs: Algumas planilhas divergem na quantidade de atributos (ex: Pessoas 2007 e 2017)

#print(len(bases["Pessoas"]["files"]))

dfPessoas = pd.read_csv(f'{url}/{bases["Pessoas"]["folder"]}/{bases["Pessoas"]["files"][10]}?raw=true', encoding='ISO-8859-1', sep=';')
dfAcidenCausas = pd.read_csv(f'{url}/{bases["Acidentes"]["folder"]}/{bases["Acidentes"]["files"][0]}?raw=true', encoding='ISO-8859-1', sep=';')

In [ ]:
dfAcidenCausas.info()
dfPessoas.info()

In [ ]:
len(dfAcidenCausas[dfAcidenCausas.duplicated(subset='pesid')])
#len(dfPessoas[dfPessoas.duplicated(subset='id')])

## Base 1: **Agrupados por ocorrência**

In [ ]:
# Observações para a reunião (Hugo)

# Tranformação dos atributos categóricos para numéricos (One-hot ou colocar números)
# Escolher atributos que podem gerar enviesamento(Naturalidade, Sexo) 
# Escolher atributos que serão estimados (Classificação e regressão)
# Normalização/padronização dos atributos numéricos
# Escolher estratégias para solução de dados faltantes (Ignorada, Não informada, (null), NULL, NaN)
# Analisar possível problema de Id's repetidos

# Base de Pessoas
# pesid repetido
#   Naturalidade = "Não Informado"
#   Marca = (null)

# Base de Ocorrencias 
# Id repetido
#   classificacao_acidente = (null)
#   condicao_metereologica = "Ignorada"

## Base 2: **Agrupados por pessoa**

# Limpeza na base de pessoas

* Atributos para **MANTER**: **id**, **pesid**, **data_inversa**, **dia_semana**, **horario**, **uf**, **br**, **km**, **municipio**, **causa_acidente**, **tipo_acidente**, **classificação_acidente**, **condicao_metereologica**, **sentido_via**, **tracado_via**, **tipo_veiculo**, **tipo_envolvido**, **estado_fisico**, **idade**, **sexo**, **ilesos**, **feridos_leves**, **feridos_graves**, **mortos**, **latitude**, **longitude**.

* Atributos para **REMOVER**: **fase_dia**, **sentido_via**, **uso_solo**, **id_veiculo**, **marca**, **ano_fabricacao_veiculo**, **regional**, **delegacia**, **uop**.




In [24]:
# IMPORTAÇÃO DAS BASES DE PESSOAS
dfs = list()
t = len(bases['Pessoas']['files'])
for i in range(t-4, t):
  dfs.append(pd.read_csv(f'{url}/{bases["Pessoas"]["folder"]}/{bases["Pessoas"]["files"][i]}?raw=true', encoding='ISO-8859-1', sep=';'))

dfPessoas = pd.concat(dfs)

In [25]:
# REMOÇÃO DE ATRIBUTOS
dfPessoas = dfPessoas.drop(columns=['fase_dia', 'sentido_via', 'uso_solo', 'id_veiculo', 'marca', 'ano_fabricacao_veiculo', 'regional', 'delegacia', 'uop'])
dfPessoas.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609882 entries, 0 to 78411
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      609882 non-null  float64
 1   pesid                   609878 non-null  float64
 2   data_inversa            609882 non-null  object 
 3   dia_semana              609882 non-null  object 
 4   horario                 609882 non-null  object 
 5   uf                      609882 non-null  object 
 6   br                      608891 non-null  float64
 7   km                      608891 non-null  object 
 8   municipio               609882 non-null  object 
 9   causa_acidente          609882 non-null  object 
 10  tipo_acidente           609882 non-null  object 
 11  classificacao_acidente  609882 non-null  object 
 12  condicao_metereologica  609882 non-null  object 
 13  tipo_pista              609882 non-null  object 
 14  tracado_via          

In [27]:
# VERIFICAÇÃO DE NULOS
def testa_valores_nulos(dataframe, atributo):
  if sum(dataframe[atributo].isnull()) == 0:
    print('\033[94m', atributo, ' => OK (não tem valores nulos)')
  else:
    print('\033[93m', atributo, ' => *** TEM VALORES NULOS')

def testa_valores_nulos(dataframe, atributo):
  if sum(dataframe[atributo] == '(null)') == 0:
    print('\033[94m', atributo, ' => OK (não tem valores "(null)")')
  else:
    print('\033[93m', atributo, ' => *** TEM VALORES "(null)"')

for attr in dfPessoas.columns:
    testa_valores_nulos(dfPessoas, attr)

 id  => OK (não tem valores nulos)
 pesid  => *** TEM VALORES NULOS
 data_inversa  => OK (não tem valores nulos)
 dia_semana  => OK (não tem valores nulos)
 horario  => OK (não tem valores nulos)
 uf  => OK (não tem valores nulos)
 br  => *** TEM VALORES NULOS
 km  => *** TEM VALORES NULOS
 municipio  => OK (não tem valores nulos)
 causa_acidente  => OK (não tem valores nulos)
 tipo_acidente  => OK (não tem valores nulos)
 classificacao_acidente  => OK (não tem valores nulos)
 condicao_metereologica  => OK (não tem valores nulos)
 tipo_pista  => OK (não tem valores nulos)
 tracado_via  => OK (não tem valores nulos)
 tipo_veiculo  => OK (não tem valores nulos)
 tipo_envolvido  => OK (não tem valores nulos)
 estado_fisico  => OK (não tem valores nulos)
 idade  => *** TEM VALORES NULOS
 sexo  => OK (não tem valores nulos)
 ilesos  => OK (não tem valores nulos)
 feridos_leves  => OK (não tem valores nulos)
 feridos_graves  => OK (não tem valores nulos)
 mortos  => OK (não tem valores nulos

## Base 3: **Agrupados por pessoa - Todas as causas e tipos de acidentes (a partir de 2017)**

# **Etapa Exploração**